In [1]:
import sys

sys.path.insert(0, '..')

import os
import requests
import json
from tempfile import mkdtemp
from urllib.request import urlretrieve
import libsbml
import amici
import os
from biosimulator_processes.utils import prepare_single_ode_process_document
from biosimulator_processes.data_model.compare_data_model import ODEComparisonDocument, DocumentFactory
from biosimulator_processes import CORE
from process_bigraph import Composite, pp

CobraProcess registered successfully.
CopasiProcess registered successfully.
_CopasiProcess registered successfully.
SmoldynProcess registered successfully.
TelluriumProcess registered successfully.
AmiciProcess registered successfully.
ODEComparatorProcess registered successfully.
GetSbmlStep registered successfully.
CompositionPlotter registered successfully.
Plotter2d registered successfully.
ODEComparatorStep registered successfully.


In [2]:
import numpy as np 

y = np.array([0.5, 1.0, 0.75, 0.60, 0.66])

average_y = np.mean(y)

# Calculate the MSE for each value in y with respect to the average
mse_scores = (y - average_y) ** 2

mse_scores, average_y

(array([0.040804, 0.088804, 0.002304, 0.010404, 0.001764]), 0.7020000000000001)

In [3]:
"""TODO: Update the input/output schema of the ODEComparatorProcess to mimic the following:

{('emitter',): [{
    'copasi': [
        {
            'floating_species_concentrations': {
                'plasminogen': 0.0,
                'plasmin': 0.0,
                'single intact chain urokinase-type plasminogen activator': 0.0,
                'two-chain urokinase-type plasminogen activator': 0.0,
                'x': 0.0,
                'x-plasmin': 0.0},
           'time': 0.0
        },
        ...
    ],
    'amici': [
        {
            'floating_species_concentrations': {
                'plasminogen': 1.1758171177387002e+16,
                'plasmin': 1096150505274.1506,
                'single intact chain urokinase-type plasminogen activator': 2955755808974603.0,
                'two-chain urokinase-type plasminogen activator': 80249.33829510311,
                'x': 0.0,
                'x-plasmin': 0.0},
           'time': 0.0},
        },
       ...
    ]


"""

"TODO: Update the input/output schema of the ODEComparatorProcess to mimic the following:\n\n{('emitter',): [{\n    'copasi': [\n        {\n            'floating_species_concentrations': {\n                'plasminogen': 0.0,\n                'plasmin': 0.0,\n                'single intact chain urokinase-type plasminogen activator': 0.0,\n                'two-chain urokinase-type plasminogen activator': 0.0,\n                'x': 0.0,\n                'x-plasmin': 0.0},\n           'time': 0.0\n        },\n        ...\n    ],\n    'amici': [\n        {\n            'floating_species_concentrations': {\n                'plasminogen': 1.1758171177387002e+16,\n                'plasmin': 1096150505274.1506,\n                'single intact chain urokinase-type plasminogen activator': 2955755808974603.0,\n                'two-chain urokinase-type plasminogen activator': 80249.33829510311,\n                'x': 0.0,\n                'x-plasmin': 0.0},\n           'time': 0.0},\n        },\n 

In [4]:
# Step 1: Define the input parameters of the api method

biomodel_id = 'BIOMD0000000630'
sbml_dirpath = '../biosimulator_processes/model_files/sbml'
model_fp = os.path.join(sbml_dirpath, f'{biomodel_id}_url.xml')
species_context = 'concentrations'
species_port_name = f'floating_species_{species_context}'
species_store = [f'floating_species_{species_context}_store']
duration = 30
n_steps = 42
simulators = ['copasi', 'tellurium']

In [5]:
def run_workflow(bio_id):
    from tempfile import mkdtemp
    from biosimulator_processes.io import fetch_sbml_file
    directory = mkdtemp()
    model_fp = fetch_sbml_file(bio_id, save_dir=directory)
    COMPOSITE_DOC = {
        'copasi_simple': {
            '_type': 'process',
              'address': 'local:copasi',
              'config': {'model': {'model_source': model_fp}},
              'inputs': {'floating_species_concentrations': ['copasi_simple_floating_species_concentrations_store'],
               'model_parameters': ['model_parameters_store'],
               'time': ['time_store'],
               'reactions': ['reactions_store']},
              'outputs': {'floating_species_concentrations': ['copasi_simple_floating_species_concentrations_store'],
               'time': ['time_store']}},
        'emitter': {
             '_type': 'step',
              'address': 'local:ram-emitter',
              'config': {
                  'emit': {
                      'copasi_simple_floating_species_concentrations': 'tree[float]',
                      'tellurium_simple_floating_species_concentrations': 'tree[float]',
                      'time': 'float'
                  }
              },
              'inputs': {
                  'copasi_simple_floating_species_concentrations': ['copasi_simple_floating_species_concentrations_store'],
                  'tellurium_simple_floating_species_concentrations': ['tellurium_simple_floating_species_concentrations_store'],
                  'time': ['time_store']
              }
        },
        'tellurium_simple': {
            '_type': 'process',
              'address': 'local:tellurium',
              'config': {'model': {'model_source': model_fp}},
              'inputs': {'floating_species_concentrations': ['tellurium_simple_floating_species_concentrations_store'],
               'model_parameters': ['model_parameters_store'],
               'time': ['time_store'],
               'reactions': ['reactions_store']},
              'outputs': {'floating_species_concentrations': ['tellurium_simple_floating_species_concentrations_store'],
               'time': ['time_store']}}}

    comp = Composite(config={'state': COMPOSITE_DOC}, core=CORE)
    comp.run(duration)
    print(model_fp)
    return comp.gather_results()

In [6]:
data = run_workflow('BIOMD0000000744')

found a filepath
/var/folders/yy/8crj8x7x5_3b86f0js6_0bn00000gr/T/tmpfrj3gueq/BIOMD0000000744.xml


/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/demos/../biosimulator_processes/processes/copasi_process.py:205: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [7]:
for values in data.values():
    for v in values:
        for name, val in v.items():
            print(f'{name}: {val}')

copasi_simple_floating_species_concentrations: {'x': 0.0, 'y': 0.0, 'z': 0.0, 'w': 0.0, 'v': 0.0}
tellurium_simple_floating_species_concentrations: {'x': 0.0, 'y': 0.0, 'z': 0.0, 'w': 0.0, 'v': 0.0}
time: 0.0
copasi_simple_floating_species_concentrations: {'x': 4.561206356842886e+29, 'y': 3.334905159717989e+27, 'z': 1.121570997999801e+29, 'w': 1.026951232159065e+30, 'v': 8.120385674906024e+29}
tellurium_simple_floating_species_concentrations: {'x': 0.0, 'y': 0.0, 'z': 3465.232864640857, 'w': 0.0, 'v': 0.0}
time: 2.0
copasi_simple_floating_species_concentrations: {'x': 8.894682689109246e+29, 'y': 6.643874349137882e+27, 'z': 2.220954275352491e+29, 'w': 2.9872793747975713e+30, 'v': 2.3928001516460923e+30}
tellurium_simple_floating_species_concentrations: {'x': 0.0, 'y': 0.0, 'z': 6861.854417330959, 'w': 0.0, 'v': 0.0}
time: 4.0
copasi_simple_floating_species_concentrations: {'x': 1.3127260459622526e+30, 'y': 9.936839371983873e+27, 'z': 3.2985891780714354e+29, 'w': 5.819505827654794e+30, '

In [8]:
sim = comp.state['tellurium_simple']['instance']


dir(sim.simulator)

import tellurium as te

results = sim.simulator.simulate(0, duration, 1000)

sim.simulator.plot(results)


NameError: name 'comp' is not defined

In [ ]:
comp = Composite(config={'state': composite_doc}, core=CORE)
comp.run(duration)
data = comp.gather_results()

In [ ]:
data

In [ ]:
def generate_comparison(duration: int, n_steps: int, simulators: list[str]):
    biomodel_id = 'BIOMD0000000744'
    comparison_doc = {
        'copasi': {
                '_type': 'process',
                'address': 'local:compare_ode_process',
                'config': {
                    'biomodel_id': biomodel_id,
                    'duration': duration,
                    'num_steps': n_steps,
                    'simulators': simulators,
                },
                'inputs': {
                    'floating_species_concentrations': ['floating_species_concentrations_store'],
                    'model_parameters': ['model_parameters_store'],
                    'time': ['time_store'],
                    'reactions': ['reactions_store']
                },
                'outputs': {
                    'floating_species_concentrations': ['floating_species_concentrations_store'],
                    'time': ['time_store']
                }
            },
            'emitter': {
                '_type': 'step',
                'address': 'local:ram-emitter',
                'config': {
                    'emit': {
                        'time': 'float',
                        'floating_species_concentrations': 'tree[float]'
                    },
                },
                'inputs': {
                    'floating_species_concentrations': ['floating_species_concentrations_store'],
                    'time': ['time_store']
                }
            }
    }
    
    wf = Composite(config={'state': comparison_doc}, core=CORE)
    wf.run(duration)
    results = wf.gather_results()
    pp(results)
    return results

In [ ]:
results = generate_comparison(duration, n_steps, simulators)

results

In [ ]:

document = {
    'sbml': {
        '_type': 'step',
        'address': 'local:get_sbml',
        'config': {
            'biomodel_id': biomodel_id
        },
        'inputs': {
                'biomodel_id': ['biomodel_id_store']
        },
        'outputs': {
                'sbml_model_fp': ['sbml_model_fp_store'],                             
        }
    },
    'ode_compare': {
        '_type': 'step',
        'address': 'local:compare_ode_step',
        'inputs': {
            'data': ['sbml_model_fp_store']
        },
        'outputs': {
            'comparison': ['comparison_store']
        }
    },
    'simulation': {
        'copasi': {
            '_type': 'process',
            'address': 'local:copasi',
            'config': {
                'model': {
                    'model_source': 'comparison_store'
                },
                'method': 'lsoda',
                'species_context': 'concentrations'
            },
            'inputs': {
                'floating_species_concentrations': ['floating_species_concentrations_store'],
                'model_parameters': ['model_parameters_store'],
                'time': ['time_store'],
                'reactions': ['reactions_store']
            },
            'outputs': {
                'floating_species_concentrations': ['floating_species_concentrations_store'],
                'time': ['time_store']
            }
        },
        'emitter': {
            '_type': 'step',
            'address': 'local:ram-emitter',
            'config': {
                'emit': {
                    'comparison': 'string',
                    'names': 'list[string]'
                },
            },
            'inputs': {
                'comparison': 'comparison_store',
                'names': ['names_store']
            }
        }
    } 
}
    


In [ ]:
class ODEComparator:
    def __init__(self, biomodel_id, dirpath, dur):
        document_a = {
            'sbml': {
                '_type': 'step',
                'address': 'local:get_sbml_step',
                'config': {
                    'biomodel_id': biomodel_id,
                    'save_dir': dirpath
                },
                'inputs': {
                        'biomodel_id': ['biomodel_id_store']
                },
                'outputs': {
                        'sbml_model_fp': ['sbml_model_fp_store'],                             
                }
            },
            'emitter_a': {
                '_type': 'step',
                'address': 'local:ram-emitter',
                'config': {
                    'emit': {
                        'comparison': 'string'}},
                'inputs': {
                    'model_fp': ['sbml_model_fp_store']}
            },
            'copasi': {
                '_type': 'process',
                'address': 'local:copasi',
                'config': {
                    'model': {
                        'model_source': ['sbml_model_fp_store']
                    },
                    'method': 'lsoda',
                    'species_context': 'concentrations',
                },
                'inputs': {
                    'floating_species_concentrations': ['floating_species_concentrations_store'],
                    'model_parameters': ['model_parameters_store'],
                    'time': ['time_store'],
                    'reactions': ['reactions_store']
                },
                'outputs': {
                    'floating_species_concentrations': ['floating_species_concentrations_store'],
                    'time': ['time_store']
                }
            },
            'emitter_b': {
                '_type': 'step',
                'address': 'local:ram-emitter',
                'config': {
                    'emit': {
                        'floating_species_concentrations': 'tree[float]'
                    },
                },
                'inputs': {
                    'floating_species_concentrations': ['floating_species_concentrations_store'],
                }
            }
        }
        
        wf = Composite(config={'state': document_a}, core=CORE)
        wf.run(dur)
        self.results = wf.gather_results()
        

In [ ]:
compare = ODEComparator(biomodel_id, mkdtemp(), 10)

compare.results

In [ ]:
def generate_composite(doc: dict) -> Composite:
    return Composite(config={'state': doc}, core=CORE)

def get_output(doc: dict, dur: int = 1) -> dict:
    comp = generate_composite(doc)
    comp.run(dur)
    return comp.gather_results()

def generate_outputs(docs: list[dict], dur: int = 1) -> list[dict]:
    outputs = []
    for doc in docs:
        result = get_output(doc, dur)
        outputs.append(result)
    return outputs 
    
def run_workflow(biomodel_id: str, dur: int = 10):
    import tempfile
    dirpath = tempfile.mkdtemp()
    document_a = {
        'sbml': {
            '_type': 'step',
            'address': 'local:get_sbml_step',
            'config': {
                'biomodel_id': biomodel_id,
                'save_dir': dirpath
            },
            'inputs': {
                    'biomodel_id': ['biomodel_id_store']
            },
            'outputs': {
                    'sbml_model_fp': ['sbml_model_fp_store'],                             
            }
        },
        'ode_compare': {
            '_type': 'step',
            'address': 'local:compare_ode_step',
            'inputs': {
                'data': ['sbml_model_fp_store']
            },
            'outputs': {
                'comparison': ['comparison_store']
            }
        },
        'emitter_a': {
            '_type': 'step',
            'address': 'local:ram-emitter',
            'config': {
                'emit': {
                    'comparison': 'string',
                },
            },
            'inputs': {
                'comparison': ['comparison_store'],
            }
        }
        
    }
    # sbml_fp = get_output(document_a, 1)[('emitter_a',)][0]['comparison']
    # print(sbml_fp)
    wf1 = Composite(config={'state': document_a}, core=CORE)
    wf1.run(1)
    sbml_fp = wf1.gather_results()[('emitter_a',)][0]['comparison']
    
    document_b = {
        'copasi': {
            '_type': 'process',
            'address': 'local:copasi',
            'config': {
                'model': {
                    'model_source': sbml_fp
                },
                'method': 'lsoda',
                'species_context': 'concentrations',
                'state': wf1.state
            },
            'inputs': {
                'floating_species_concentrations': ['floating_species_concentrations_store'],
                'model_parameters': ['model_parameters_store'],
                'time': ['time_store'],
                'reactions': ['reactions_store']
            },
            'outputs': {
                'floating_species_concentrations': ['floating_species_concentrations_store'],
                'time': ['time_store']
            }
        },
        'emitter_b': {
            '_type': 'step',
            'address': 'local:ram-emitter',
            'config': {
                'emit': {
                    'floating_species_concentrations': 'tree[float]'
                },
            },
            'inputs': {
                'floating_species_concentrations': ['floating_species_concentrations_store'],
            }
        }
    }
    
    print(sbml_fp)
    wf = Composite(config={'state': document_b}, core=CORE)
    wf.run(dur)
    results = wf.gather_results()
    pp(results)
    return results 
    

In [ ]:
results = run_workflow(biomodel_id, duration)

results

In [ ]:
wf = Composite(config={'state': document}, core=CORE)

In [ ]:
wf.run(10)
state = wf.gather_results()

In [ ]:
state

In [ ]:
state

In [ ]:
from biosimulator_processes.io import fetch_sbml_file
sbml_file = fetch_sbml_file(biomodel_id)

In [ ]:
sbml_file

In [ ]:
from basico import load_model, get_species

model = load_model(sbml_file)


model

In [ ]:
get_species(model=model)